In [0]:
from google.colab import drive
drive.mount('/warwick_qu_dataset')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /warwick_qu_dataset


In [0]:
!wget https://warwick.ac.uk/fac/sci/dcs/research/tia/glascontest/download/warwick_qu_dataset_released_2016_07_08.zip

--2019-09-12 01:05:25--  https://warwick.ac.uk/fac/sci/dcs/research/tia/glascontest/download/warwick_qu_dataset_released_2016_07_08.zip
Resolving warwick.ac.uk (warwick.ac.uk)... 137.205.28.41
Connecting to warwick.ac.uk (warwick.ac.uk)|137.205.28.41|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 180902609 (173M) [application/zip]
Saving to: ‘warwick_qu_dataset_released_2016_07_08.zip’

warwick_qu_dataset_ 100%[===================>] 172.52M  10.4MB/s    in 32s     

2019-09-12 01:05:58 (5.45 MB/s) - ‘warwick_qu_dataset_released_2016_07_08.zip’ saved [180902609/180902609]



In [0]:
!ls '/warwick_qu_dataset'

'My Drive'


In [0]:
! unzip -q 'warwick_qu_dataset_released_2016_07_08.zip'

In [0]:
from PIL import Image
import PIL
import os
dirname = "/content/Warwick QU Dataset (Released 2016_07_08)/"
reshapedirname = "./reshaped_warwick/"
final_height = 400
final_width = 600
try:
	os.stat(reshapedirname)
except:
	os.mkdir(reshapedirname)
for subdirname in ["segments_train","segments_test","segments_train_eval","images_train","images_test","images_train_eval"]:
	try:
		os.stat(reshapedirname+subdirname)
	except:
		os.mkdir(reshapedirname+subdirname)
prefix_name = ["images","segments"]
suffix_name = ["train","test"]
for file in os.listdir(dirname):
	if file.endswith(".bmp"):
		img = Image.open(dirname+file)
		width, height = img.size
		istest,isanno = False,False
		if file[:4] == "test":
			istest = True
		if file.find("anno") != -1:
			isanno = True
		if isanno:
			# change the image to saturated
			img = img.point(lambda i: i * 255)
		reshaped_image = img = img.resize((final_width, final_height), PIL.Image.ANTIALIAS)
		newfilename = reshapedirname+prefix_name[isanno]+"_"+suffix_name[istest]+"/"+file.split('.')[0]
		newfilenamefull = newfilename+".png"
		reshaped_image.save(newfilenamefull,"PNG")

In [0]:
from data_input import gen_csv_paths

gen_csv_paths('reshaped_warwick/', 'train', 0)
gen_csv_paths('reshaped_warwick/', 'train', 1)
gen_csv_paths('reshaped_warwick/', 'train', 2)
gen_csv_paths('reshaped_warwick/', 'train', 3)
gen_csv_paths('reshaped_warwick/', 'test')
gen_csv_paths('reshaped_warwick/', 'train_eval')

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import time
import logging
import tensorflow as tf

import mainutils

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', '/tmp/warwick_train',
						   """Directory where to write event logs """
						   """and checkpoint.""")
tf.app.flags.DEFINE_string('eval_data', 'train',
						   """Either 'test' or 'train'.""")
tf.app.flags.DEFINE_integer('max_steps', 4000,
							"""Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
							"""Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 200,
							"""How often to log results to the console.""")
tf.logging.set_verbosity(tf.logging.DEBUG)

def train(sessid):
	with tf.Graph().as_default():
		ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
		global_step_init = -1
		global_step = tf.contrib.framework.get_or_create_global_step()
		if ckpt and ckpt.model_checkpoint_path:
			# Restores from checkpoint
			global_step_init = int(ckpt.model_checkpoint_path.split('/')[-1]
								   .split('-')[-1])

		
		images, labels, i_paths = mainutils.inputs(eval_data=FLAGS.eval_data,sessid=sessid)
		
		s_fuse, encoding = mainutils.inference_bottleneck(images)
		mainutils.get_show_preds(s_fuse)



		# Calculate loss.
		segments_labels = labels
		with tf.variable_scope('{}_cross_entropy'.format('s')) as scope:
			class_prop = mainutils.S_CLASS_PROP
			weight_per_label = tf.scalar_mul(class_prop, tf.cast(tf.equal(segments_labels, 0),
																 tf.float32)) + \
							   tf.scalar_mul(1.0 - class_prop, tf.cast(tf.equal(segments_labels, 1),
																	   tf.float32))
			cross_entropy = tf.losses.sparse_softmax_cross_entropy(
				labels=tf.squeeze(segments_labels, squeeze_dims=[3]), logits=s_fuse)
			cross_entropy_weighted = tf.multiply(weight_per_label, cross_entropy)
			cross_entropy_mean = tf.reduce_mean(cross_entropy_weighted, name=scope.name)

		loss = cross_entropy_mean

		# Build a Graph that trains the model with one batch of examples and
		# updates the model parameters.
		train_op = mainutils.train(loss, global_step)

		class _LoggerHook(tf.train.SessionRunHook):
			"""Logs loss and runtime."""

			def begin(self):
				self._step = global_step_init
				self._start_time = time.time()

			def before_run(self, run_context):
				self._step += 1
				return tf.train.SessionRunArgs(loss)  # Asks for loss value.

			def after_run(self, run_context, run_values):
				if self._step % FLAGS.log_frequency == 0:
					current_time = time.time()
					duration = current_time - self._start_time
					self._start_time = current_time

					loss_value = run_values.results
					examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
					sec_per_batch = float(duration / FLAGS.log_frequency)

					format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
								  'sec/batch)')
					print(format_str % (datetime.now(), self._step, loss_value,
										examples_per_sec, sec_per_batch))

		saver = tf.train.Saver()
		with tf.train.MonitoredTrainingSession(
				checkpoint_dir=FLAGS.train_dir,
				hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
					   tf.train.NanTensorHook(loss),
					   _LoggerHook()],
				config=tf.ConfigProto(
					log_device_placement=FLAGS.log_device_placement)
				) as mon_sess:
			ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)

			if ckpt:
				saver.restore(mon_sess, ckpt.model_checkpoint_path)
				logging.info("Model restored from file: %s" % ckpt.model_checkpoint_path)
			while not mon_sess.should_stop():
				_,losseval = mon_sess.run([train_op,loss])

def main(argv=None):  # pylint: disable=unused-argument
	FLAGS.train_dir = '/tmp/warwick_train_0'
	train(0)
	FLAGS.train_dir = '/tmp/warwick_train_1'
	train(1)
	FLAGS.train_dir = '/tmp/warwick_train_2'
	train(2)
	FLAGS.train_dir = '/tmp/warwick_train_3'
	train(3)

if __name__ == '__main__':
	tf.app.run()

W0912 01:08:04.647044 140003161491328 deprecation.py:323] From <ipython-input-7-10441686e255>:32: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0912 01:08:04.666014 140003161491328 deprecation.py:323] From data_input.py:119: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0912 01:08:04.676755 140003161491328 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/training/input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will b

2019-09-12 01:08:43.065151: step 0, loss = 0.76 (7.0 examples/sec; 0.142 sec/batch)


W0912 01:08:45.760337 140003161491328 basic_session_run_hooks.py:724] It seems that global step (tf.train.get_global_step) has not been increased. Current value (could be stable): 0 vs previous value: 0. You could increase the global step by passing tf.train.get_global_step() to Optimizer.apply_gradients or Optimizer.minimize.
I0912 01:08:52.533031 140003161491328 basic_session_run_hooks.py:692] global_step/sec: 10.5592
I0912 01:08:58.047202 140003161491328 basic_session_run_hooks.py:692] global_step/sec: 18.1352


2019-09-12 01:08:58.057214: step 200, loss = 0.23 (13.3 examples/sec; 0.075 sec/batch)


I0912 01:09:03.532685 140003161491328 basic_session_run_hooks.py:692] global_step/sec: 18.2299
I0912 01:09:09.031238 140003161491328 basic_session_run_hooks.py:692] global_step/sec: 18.1867


...

2019-09-12 01:25:20.511581: step 3800, loss = 0.04 (17.8 examples/sec; 0.056 sec/batch)


I0912 01:25:26.117115 140003161491328 basic_session_run_hooks.py:692] global_step/sec: 17.8095
I0912 01:25:31.730911 140003161491328 basic_session_run_hooks.py:692] global_step/sec: 17.8133
I0912 01:25:31.744002 140003161491328 basic_session_run_hooks.py:606] Saving checkpoints for 4000 into /tmp/warwick_train_3/model.ckpt.


2019-09-12 01:25:31.743053: step 4000, loss = 0.04 (17.8 examples/sec; 0.056 sec/batch)


SystemExit: ignored

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import math
import time
from PIL import Image
import numpy as np
import tensorflow as tf

import mainutils

FLAGS = tf.app.flags.FLAGS

# tf.app.flags.DEFINE_string('eval_dir', '/tmp/warwick_eval',
# 						   """Directory where to write event logs.""")
# tf.app.flags.DEFINE_string('eval_data', 'train_eval',
# 						   """Either 'test' or 'train' or 'train_eval'.""")
# tf.app.flags.DEFINE_string('checkpoint_dir', '/tmp/warwick_train/',
# 						   """Directory where to read model checkpoints.""")
# tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
# 							"""How often to run the eval.""")
# tf.app.flags.DEFINE_integer('num_examples', 100,
# 							"""Number of examples to run.""")
# tf.app.flags.DEFINE_boolean('run_once', True,
# 							"""Whether to run eval only once.""")


def eval_once(saver, dice_op, summary_writer, summary_op, s_fuse, images, labels, i_paths, encoding, sessid):
	"""Run Eval once.
	Args:
		saver: Saver.
		summary_writer: Summary writer.
		summary_op: Summary op.
	"""

	FLAGS.checkpoint_dir = '/tmp/warwick_train_'+str(sessid)
	with tf.Session() as sess:
		ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
		if ckpt and ckpt.model_checkpoint_path:
			# Restores from checkpoint
			saver.restore(sess, ckpt.model_checkpoint_path)
			print("Model restored from file: %s" % ckpt.model_checkpoint_path)
			global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
		else:
			print('No checkpoint file found')
			return

		# Start the queue runners.
		coord = tf.train.Coordinator()
		predictions = {}
		encodings = {}
		dice_scores = {}
		try:
			threads = []
			for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
				threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
												 start=True))

			num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
			avg_s_dice = 0
			step = 0
			while step < num_iter and not coord.should_stop():
				s_dice,i_path,s_fuse_out,encoded_image = sess.run([dice_op,i_paths,s_fuse,encoding])
				predictions[i_path[0]] = s_fuse_out
				encodings[i_path[0]] = encoded_image
				dice_scores[i_path[0]] = s_dice
				im0 = s_fuse_out[0,:,:,0]
				im1 = s_fuse_out[0,:,:,1]
				image = (im1>im0)*128
				im = Image.fromarray(image.astype(np.uint8))
        
				im.save('reshaped_warwick/results/'+i_path[0].split('/')[2])
				avg_s_dice += s_dice
				step += 1

			avg_s_dice /= step
			print('%s: s_dice avg = %.3f' % (datetime.now(), avg_s_dice))

			summary = tf.Summary()
			summary.ParseFromString(sess.run(summary_op))
			summary.value.add(tag='dice_s', simple_value=avg_s_dice)
			summary_writer.add_summary(summary, global_step)
		except Exception as e:  # pylint: disable=broad-except
			coord.request_stop(e)
		if FLAGS.eval_data == 'train_eval':
			np.save('train_eval_data_'+str(sessid)+'.npy',[predictions, encodings, dice_scores]) # otherwise don't save
		coord.request_stop()
		coord.join(threads, stop_grace_period_secs=20)




In [38]:
def evaluate():
	
	for sessid in range(4):
		FLAGS.eval_dir = '/tmp/warwick_eval_' + str(sessid)
		with tf.Graph().as_default() as g:
			# Get images and labels for BBBC006.
			images, labels, i_paths = mainutils.inputs(eval_data=FLAGS.eval_data, sessid=sessid)
			
			# Build a Graph that computes the logits predictions from the
			
			s_fuse, encoding = mainutils.inference_bottleneck(images, train=False)

			dice_op = mainutils.dice_op(s_fuse, labels)

			# Restore the moving average version of the learned variables for eval.
			variable_averages = tf.train.ExponentialMovingAverage(
				mainutils.MOVING_AVERAGE_DECAY)
			variables_to_restore = variable_averages.variables_to_restore()
			saver = tf.train.Saver(variables_to_restore)

			# Build the summary operation based on the TF collection of Summaries.
			summary_op = tf.summary.merge_all()
			summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)
			s_fuse_softmax = tf.nn.softmax(s_fuse)

			while True:
				eval_once(saver, dice_op, summary_writer, summary_op, s_fuse_softmax, images, labels, i_paths, encoding, sessid)
				if FLAGS.run_once:
					break
				time.sleep(FLAGS.eval_interval_secs)


def main(argv=None):  # pylint: disable=unused-argument
	if tf.gfile.Exists(FLAGS.eval_dir):
		tf.gfile.DeleteRecursively(FLAGS.eval_dir)
	tf.gfile.MakeDirs(FLAGS.eval_dir)
	evaluate()


if __name__ == '__main__':
	tf.app.run()

I0912 02:06:06.202944 140003161491328 saver.py:1280] Restoring parameters from /tmp/warwick_train_0/model.ckpt-4000


Model restored from file: /tmp/warwick_train_0/model.ckpt-4000


I0912 02:06:07.208344 140003161491328 coordinator.py:224] Error reported to Coordinator: <type 'exceptions.IOError'>, [Errno 2] No such file or directory: 'reshaped_warwick/results/train_30.png'


IOError: ignored

In [33]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import math
import time
from PIL import Image
import numpy as np
import tensorflow as tf

import mainutils

# FLAGS = tf.app.flags.FLAGS

# tf.app.flags.DEFINE_string('eval_dir', '/tmp/warwick_eval',
# 						   """Directory where to write event logs.""")
# tf.app.flags.DEFINE_string('eval_data', 'train_eval',
# 						   """Either 'test' or 'train' or 'train_eval'.""")
# tf.app.flags.DEFINE_string('checkpoint_dir', '/tmp/warwick_train',
# 						   """Directory where to read model checkpoints.""")
# tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
# 							"""How often to run the eval.""")
# tf.app.flags.DEFINE_integer('num_examples', 100,
# 							"""Number of examples to run.""")
# tf.app.flags.DEFINE_boolean('run_once', True,
# 							"""Whether to run eval only once.""")


def eval_once(saver, dice_op, summary_writer, summary_op, s_fuse, images, labels, i_paths, encoding, sessid):
	"""Run Eval once.
	Args:
		saver: Saver.
		summary_writer: Summary writer.
		summary_op: Summary op.
	"""

	FLAGS.checkpoint_dir = '/tmp/warwick_train_'+str(sessid)
	with tf.Session() as sess:
		ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
		if ckpt and ckpt.model_checkpoint_path:
			# Restores from checkpoint
			saver.restore(sess, ckpt.model_checkpoint_path)
			print("Model restored from file: %s" % ckpt.model_checkpoint_path)
			global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
		else:
			print('No checkpoint file found')
			return

		# Start the queue runners.
		coord = tf.train.Coordinator()
		predictions = {}
		encodings = {}
		dice_scores = {}
		try:
			threads = []
			for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
				threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
												 start=True))

			num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
			avg_s_dice = 0
			step = 0
			while step < num_iter and not coord.should_stop():
				s_dice,i_path,s_fuse_out,encoded_image = sess.run([dice_op,i_paths,s_fuse,encoding])
				predictions[i_path[0]] = s_fuse_out
				encodings[i_path[0]] = encoded_image
				dice_scores[i_path[0]] = s_dice
				im0 = s_fuse_out[0,:,:,0]
				im1 = s_fuse_out[0,:,:,1]
				image = (im1>im0)*128
				im = Image.fromarray(image.astype(np.uint8))
				im.save('results/'+i_path[0].split('/')[2]+'.bmp')
				avg_s_dice += s_dice
				step += 1

			avg_s_dice /= step
			print('%s: s_dice avg = %.3f' % (datetime.now(), avg_s_dice))

			summary = tf.Summary()
			summary.ParseFromString(sess.run(summary_op))
			summary.value.add(tag='dice_s', simple_value=avg_s_dice)
			summary_writer.add_summary(summary, global_step)
		except Exception as e:  # pylint: disable=broad-except
			coord.request_stop(e)
		if FLAGS.eval_data == 'train_eval':
			np.save('train_eval_data_'+str(sessid)+'.npy',[predictions, encodings, dice_scores]) # otherwise don't save
		coord.request_stop()
		coord.join(threads, stop_grace_period_secs=20)


def evaluate():
	"""Eval BBBC006 for a number of steps."""
	for sessid in range(4):
		FLAGS.eval_dir = '/tmp/warwick_eval_/' + str(sessid)
		with tf.Graph().as_default() as g:
			# Get images and labels for BBBC006.
			images, labels, i_paths = mainutils.inputs(eval_data=FLAGS.eval_data, sessid=sessid)
			
			# Build a Graph that computes the logits predictions from the
			# inference model.
			# s_fuse = mainutils.inference(images, train=False)
			s_fuse, encoding = mainutils.inference_bottleneck(images, train=False)

			dice_op = mainutils.dice_op(s_fuse, labels)

			# Restore the moving average version of the learned variables for eval.
			variable_averages = tf.train.ExponentialMovingAverage(
				mainutils.MOVING_AVERAGE_DECAY)
			variables_to_restore = variable_averages.variables_to_restore()
			saver = tf.train.Saver(variables_to_restore)

			# Build the summary operation based on the TF collection of Summaries.
			summary_op = tf.summary.merge_all()
			summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)
			s_fuse_softmax = tf.nn.softmax(s_fuse)

			while True:
				eval_once(saver, dice_op, summary_writer, summary_op, s_fuse_softmax, images, labels, i_paths, encoding, sessid)
				if FLAGS.run_once:
					break
				time.sleep(FLAGS.eval_interval_secs)


def main(argv=None):  # pylint: disable=unused-argument
	if tf.gfile.Exists(FLAGS.eval_dir):
		tf.gfile.DeleteRecursively(FLAGS.eval_dir)
	tf.gfile.MakeDirs(FLAGS.eval_dir)
	evaluate()


if __name__ == '__main__':
	tf.app.run()

I0912 02:00:17.474838 140003161491328 saver.py:1280] Restoring parameters from /tmp/warwick_train_0/model.ckpt-4000


Model restored from file: /tmp/warwick_train_0/model.ckpt-4000


I0912 02:00:18.423402 140003161491328 coordinator.py:224] Error reported to Coordinator: <type 'exceptions.IOError'>, [Errno 2] No such file or directory: 'results/train_30.png.bmp'


IOError: ignored

In [0]:
import numpy as np
from scipy import spatial
import operator

predictions, encodings, dice_scores = np.array([None]*4), np.array([None]*4), np.array([None]*4)
for i in range(4):
	predictions[i], encodings[i], dice_scores[i] = np.load('train_eval_data_'+str(i)+'.npy')

keys = dice_scores[0].keys()
K, k = 20, 10

scores = {}
for key in keys:
	total_array = np.array([
			predictions[0][key],
			predictions[1][key],
			predictions[2][key],
			predictions[3][key]
		])
	scores[key] = np.sum(np.var(total_array,axis=0))
# scores list would have images tuples in decreasing order of variance
scores_list = sorted(scores.items(), key=lambda x: -x[1])
options_available = [scores_list[i][0] for i in range(K)]

def similarity(encodings, key1, key2): # cosine similarity
	similarities = [None]*4
	for i in range(4):
		similarities[i] = 1 - spatial.distance.cosine(np.mean(encodings[i][key1],axis=3).flatten(),np.mean(encodings[i][key2],axis=3).flatten())
	return np.mean(similarities)

def unit_F(similarities,key,images_selected):
	max_sim = float('-inf')
	for image_selected in images_selected:
		max_sim = max(max_sim,similarities[(key,image_selected)])
	return max_sim

def calc_F(similarities,keys,images_selected):
	F = 0
	for key in keys:
		F += unit_F(similarities,key,images_selected)
	return F

similarities = {}
for key1 in keys:
	for key2 in keys:
		similarities[(key1,key2)] = similarity(encodings,key1,key2)

# select first 20 images with highest uncertainity and then select
# the 10 representative out of them
images_selected = []
while len(images_selected) < k:
	F_options = {}
	for option in options_available:
		F_options[option] = calc_F(similarities,keys,images_selected+[option])
	best_option = max(F_options.iteritems(), key=operator.itemgetter(1))[0]
	options_available.remove(best_option)
	images_selected.append(best_option)
print(images_selected)